In [2]:
#training step
!python train.py --data_dir ../data/ \
                 --train_file SEC_Export_Final.csv \      #file name with extension. 
                 --model_dir ../models/ \   #export model artifcats path
                 --use_social no \    # just social flags
                 --spacy_features no  #spacy features  yes /no flag

Input file dimension :  (35249, 3)
#Unique Intents :  570
Unique Intents after preprocessing : 455

Running preprocessing ...
Preprocessing completed successfully

Running featurizer ...
Saving TF-IDF vectors to ../temp/tfidf_pre_trained_vectors.pkl
Featurization process completed successfully
(33860, 68124)

Training data feature dimension: (33860, 68124)
Training labels dimension: (33860, 1)

Training Logistic Regression Model for 455 Intents
Saving trained logistic_regression model  to ../temp/logistic_regression_model.pkl

Training Process Completed Successfully


In [4]:
# evaluation mode when truth labels are provided in test data. 
!python validate.py --data_dir ../data/ \
                 --test_file f2.csv \
                 --model_dir ../models/ \
                 --use_social no \
                 --spacy_features yes \
                 --tag unlabelled   ## uncomment this for batch inference, when truth labels bot are provided 

Input file dimension :  (27958, 8)

Running preprocessing ...
Preprocessing completed successfully

Running featurizer ...
Featurization process completed successfully
(27958, 59376)
Test data feature dimension: (27958, 59376)

Running Model Predictions ..
Model predictions Done


In [2]:
import nltk
##t1=t.sample(frac=0.5,replace=False,random_state=100)
#print(t1.shape)
import re
from nltk.stem import WordNetLemmatizer 
lemmatizer = nltk.stem.WordNetLemmatizer()
brackets_re = re.compile(r"[(\[].*?[)\]]")
replace_by_space_re = re.compile(r"[{}|@,;]")
non_alphanum_re = re.compile(r"[^0-9a-z#+_]")

def clean_text(text, spell_check=False):
    text = str(text).lower()  # lowercase text
    # regex clean operations
    text = re.sub(brackets_re, "", text)  # remove [] & () brackets
    text = re.sub(replace_by_space_re, " ", text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(non_alphanum_re, " ", text)  # delete symbols which are not alphanumeric numbers from text
    #text=re.sub(re.sub(re.sub(re.sub(re.sub(re.sub("%20", " ",text), "%7c",text), "|",text), "%2f",text), "/",text), "%3a", "")
    text=str(text).replace("redacted","")
    #text=str(text).replace('xe2 x80 x99t',"")
    text=str(text).replace('xe2',"")
    text=str(text).replace('x80',"")
    text=str(text).replace('x99t',"")
    #text = re.sub(r"\s{2,}", " ", text)
    # in-house financial spell checker
    if spell_check:
        text = " ".join(sc.correction(sc.tokenize(text)))
    #lemmatization
    text = " ".join([lemmatizer.lemmatize(w) for w in text.split(" ")])
    #text=" ".join([word for word in text.split(" ") if word not in stopwords.words('english')])
    return text
# t1['sent']=t1.message_text.apply(clean_text)
# t1=t1[['sent']]
# print(t1.shape)
#t1.to_csv("../data/clean_set.csv",index=False)

In [20]:
import pandas as pd,os,sys,numpy
import re
from nltk.corpus import stopwords

ds=pd.read_csv("/Users/a656526/Documents/livechats/ds.tsv",sep="\t")
def clean(text):
    text=str(text).lower()
    text=re.sub( "%20", " ",text)
    text=re.sub( "%7c", " ",text)
    text=re.sub( "%2f", " ",text)
    text=re.sub( "%3a", " ",text)
    text=re.sub( "%26", " ",text)
    text=re.sub( "%2c", " ",text)
    text=re.sub( "%3f", " ",text)
#     text=re.sub( "/", " ",text)
#     text=re.sub( ",", " ",text)
#     text=re.sub( ":", " ",text)
#     text=re.sub( "|", " ",text)
    return text
ds['clean_text']=ds._c1.apply(clean)
ds['clean_text']=ds.clean_text.apply(clean_text)
ds['sent']=ds['clean_text'].str.strip()
ds['len']=ds.sent.str.split(" ").map(len)

In [21]:
#ds[ds._c1.notnull()].shape
print(ds.clean_text.nunique())
print(ds.mid.nunique())
#round(ds.len.describe())
numpy.percentile(ds.len,95)

525236
1269313


4.0

In [22]:
import pandas as pd
msg=pd.read_csv("/Users/a656526/Documents/livechats/msgs_2020.tsv",sep="\t",error_bad_lines=False,warn_bad_lines=False)
mid=pd.read_csv("/Users/a656526/Documents/livechats/mids_2020.tsv",sep="\t",error_bad_lines=False,warn_bad_lines=False)
print(msg.columns)
msg.columns=msg.columns.str.replace("nuance_messages_masked.","")
# mid=mid[mid.metrics.notnull()]
msg=msg[msg.message_text.notnull()]
# #mid=mid[mid.tags.notnull()]

mid.drop_duplicates(['conv_id','mid'],inplace=True)
# mid['chat_flag']=(mid.metrics.str.contains('Chat'))*1

va_mid=msg.merge(mid[['conv_id','mid']],on='conv_id',how='inner')
va_mid=va_mid[va_mid.message_text.notnull()]

va_mid.shape
va_mid['sent']=va_mid.message_text.apply(clean_text)
va_mid1['sent']=va_mid1['sent'].str.strip()
va_mid['len']=va_mid.sent.str.split(" ").apply(len)

Index(['conv_id', 'message_text'], dtype='object')


In [10]:
ds1=ds[ds.mid.isin(va_mid.mid)]
va_mid1=va_mid[va_mid.mid.isin(ds.mid.unique())]
va_mid1.head()

,conv_id,message_text,mid
0,4476e01e-6991-40a2-a453-ea9489b02c46,loan,ee9c46996645ef11dcbc9e9bc8b53daa77
1,4476e01e-6991-40a2-a453-ea9489b02c46,can i request a loan online?,ee9c46996645ef11dcbc9e9bc8b53daa77
2,e0b0e88a-152c-4c29-bf43-9ac8aa7b9f4a,can i request a loan online?,eee0d5f2e67c0c11d6b90f9bc8b54aaa77
3,e467e0bd-4b4a-48f1-ac74-f2767d75ef68,can i rollover my pension to an ira?,eea0f42a5a6e1f11d98d589bc8b54aaa77
13,656a3ce5-6f16-4017-a17f-f99cf49f38bb,can i request a loan online?,ee5e7a69690a8820162000e64500d5aa33


In [23]:
print(va_mid1.mid.nunique())
print(ds1.mid.nunique())

227324
227324


In [24]:
i1=pd.DataFrame(ds.groupby(['len']).size())
i1.reset_index(inplace=True)
i1.columns=[ 'length', 'freq']
print(i1.shape)
i1.head()

(39, 2)


,length,freq
0,1,2590989
1,2,1697604
2,3,637271
3,4,227433
4,5,125812


In [43]:
va_mid.loc[va_mid.len>28,"len"]=28
round(ds.len.value_counts(normalize=True)*100,1).sort_index()
#round(va_mid.len.value_counts(normalize=True)*100,1).sort_index()

1     48.0
2     31.4
3     11.8
4      4.2
5      2.3
6      1.0
7      0.5
8      0.3
9      0.2
10     0.1
11     0.1
12     0.0
13     0.0
14     0.0
15     0.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
27     0.0
30     0.0
33     0.0
34     0.0
38     0.0
44     0.0
46     0.0
47     0.0
50     0.0
56     0.0
57     0.0
60     0.0
63     0.0
69     0.0
Name: len, dtype: float64

In [86]:
i1=pd.DataFrame(ds1.groupby(['sent']).size())
i1.reset_index(inplace=True)
i1.columns=[ 'sent', 'freq']
print(i1.shape)
i1=i1[i1.freq>500]
i1=i1.sort_values('freq',ascending=False)
print(i1.shape)
i1.to_csv("/Users/a656526/Documents/integration/ds_strings.csv",index=False)

(176293, 2)
(253, 2)


In [87]:
i1.head()

,sent,freq
96549,loan,51277
172211,withdraw money,13087
172398,withdrawal,12412
96689,loan available,11151
156084,transfer tracker,9550


In [84]:
va_mid1['len']=va_mid1['sent'].str.split(" ").map(len)
va_mid1=va_mid1[va_mid1.len>2]
i1=pd.DataFrame(va_mid1.groupby(['sent']).size())
i1.reset_index(inplace=True)
i1.columns=[ 'sent', 'freq']
print(i1.shape)
i1=i1[i1.freq>75]
i1=i1.sort_values('freq',ascending=False)
print(i1.shape)
i1.to_csv("/Users/a656526/Documents/integration/va_strings.csv",index=False)

(320332, 2)
(184, 2)


In [4]:
import pandas as pd,os,sys,numpy

from nltk.corpus import stopwords
t=pd.read_csv("../data/msgs_2020.tsv",sep="\t")
t['message_text']=t['message_text'].astype(str)
t=t[t.message_text.notnull()]

t['len']=t.message_text.str.split(" ").apply(len)
print(t.shape)
t=t[(t.len>2) & (t.len<=15)]
#del t['len']
#t.reset_index(inplace=True,drop=True)
print(t.shape)

# t1=t.loc[:round(t.shape[0]/2),:]
# t2=t.loc[round(t.shape[0]/2)+1:,:]

# t1=t1[t1.message_text.notnull()]
# t2=t2[t2.message_text.notnull()]
# print(t1.shape)
# print(t2.shape)
# t1=t1[['message_text']]
# t2=t2[['message_text']]
# t1.to_csv("../data/set1.csv",index=False)
# t2.to_csv("../data/set2.csv",index=False)

(9238311, 2)
(6728344, 2)


In [165]:
# evaluation mode when truth labels are provided in test data. 
!python validate.py --data_dir ../data/ \
                 --test_file clean_set.csv \
                 --model_dir ../temp/ \
                 --use_social no \
                 --spacy_features no \
                 --tag unlabelled   ## uncomment this for batch inference, when truth labels bot are provided 


Running preprocessing ...
Preprocessing completed successfully

Running featurizer ...
Featurization process completed successfully
(3300134, 49081)

Test data feature dimension: (3300134, 49081)

Running Model Predictions ..

Model predictions Done


In [8]:
import pandas as pd,os,sys,numpy
#inp=pd.read_csv("../output/predicted_clean_set.csv")
inp=pd.read_csv("../data/clean_set.csv")
inp['sent1']=inp.sent.str.strip()

In [14]:
i1=pd.DataFrame(inp.groupby(['sent1']).size())
i1.reset_index(inplace=True)
i1.columns=[ 'sent1', 'freq']
print(i1.shape)
i1=i1[i1.freq>200]
i1=i1.sort_values('freq',ascending=False)
print(i1.shape)
i1.to_csv("../output/strings_with_stopwords.csv",index=False)

(1826867, 2)
(517, 2)


In [55]:
from inference import *
print(run_inference( '{"utterance":"I want to downlaod my tax form"}',model_dir="../temp/",spacy_features="yes"))

#print(run_inference( '{"utterance":"I want to know my account balance"}'))

#print(run_inference('{}'))

#print(run_inference('{"utterance":" withdraw money"}'))


Running preprocessing ...
Preprocessing completed successfully

Running featurizer ...
{
   "statusCode": 400,
   "data": [
      {
         "intent1": "",
         "score1": 0,
         "tag": ""
      },
      {
         "intent2": "",
         "score2": 0,
         "tag": ""
      },
      {
         "intent3": "",
         "score3": 0,
         "tag": ""
      }
   ],
   "message": "Error occured in Preprocess/TF-IDF/Inference stage"
}


In [ ]:
import pickle
pik="/Users/a656526/Documents/livechats/data.pkl"
f=open(pik,"rb")
chat3=pickle.load(f)
f.close()
chat3.shape

In [ ]:
chat3=chat3[chat3.msgtext.notnull()]
chat3=chat3[chat3.message_text.notnull()]
chat3['len1']=chat3.msgtext.str.split(" ").apply(len)
chat3['len2']=chat3.message_text.str.split(" ").apply(len)
chat3=chat3[(chat3.len1>2) & (chat3.len1<20)]
chat3=chat3[(chat3.len2>2) & (chat3.len2<20)]
chat3.shape

In [ ]:
chat_msg=chat3[['msgtext']]
va_msg=chat3[['message_text']]
chat_msg.to_csv("../../../livechats/chat_msg.csv",index=False)
va_msg.to_csv("../../../livechats/va_msg.csv",index=False)

In [ ]:
!python validate.py  --data_dir ../../../livechats/ \
                     --test_file chat_msg.csv \
                     --model_dir ../models/ \
                     --use_social no \
                     --spacy_features no \
                     --tag unlabelled
        

In [ ]:
import pandas as pd
va=pd.read_csv("../output/va_msg.csv")
chat=pd.read_csv("../output/chat_msg.csv")
chat=chat[chat.display_pred!="I want to contact you"]
out=pd.concat([va,chat],axis=1,ignore_index=True)
out.columns=["va_intent","va_score","chat_intent","chat_score"]

#out.shape

In [ ]:
list1=pd.DataFrame(out.va_intent.value_counts()[:61]).reset_index()
list1.columns=['va_intent','freq']
list2=pd.DataFrame(out.chat_intent.value_counts()[:61]).reset_index()
list2.columns=['chat_intent','freq']
out=out[(out.chat_intent.isin(list2.chat_intent)) ]
out=out[(out.va_intent.isin(list1.va_intent))]
print(out.shape)
out.to_csv("../output/matrix.csv",index=False)

In [ ]:
import pandas as pd
#pd.pivot_table(out,index="va_intent",columns="chat_intent",aggfunc="count")
out1=pd.DataFrame(pd.crosstab(index=out['va_intent'], columns=out['chat_intent']))
out1.to_csv("../output/matrix.csv")


In [ ]:
#out1.style.background_gradient(cmap='Reds',axis=1)

In [ ]:
from preprocess import *
from featurizer import *
import argparse
import warnings
warnings.filterwarnings("ignore")

clean_data=run_preprocess(   data_dir="../data/", \
                                 filename="all_data.csv",\
                                 colnames=['Sentence','Intent','display_column']
                                 
                         )
# call featurizer
train_features,y_train=run_featurizer(clean_data,train_eval_predict="train",model_dir="../models")
print('\nTraining data feature dimension: {}'.format(train_features.shape))
print('Training labels dimension: {}'.format(y_train.shape))


In [ ]:
import sklearn
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold,ShuffleSplit

#skfold = StratifiedKFold(n_splits=4, random_state=100)
skfold=ShuffleSplit(n_splits=5, test_size=0.1, random_state=100)
model1=LogisticRegression(random_state=100,n_jobs=-1, max_iter=100,multi_class= 'ovr',fit_intercept=False,C=10,\
                        dual=True,solver='liblinear')

results_skfold = model_selection.cross_val_score(model1, train_features,y_train, cv=skfold,n_jobs=-1)
print("Accuracy: %.2f%%" % (results_skfold.mean()*100.0))

In [ ]:
model1=LogisticRegression(random_state=100,n_jobs=-1, max_iter=100,multi_class= 'ovr',fit_intercept=False,C=10,\
                        dual=True,solver='liblinear')
model1.fit(train_features,y_train)

In [ ]:
import pickle
model_output = os.path.join("../models/", "logistic_regression_model.pkl")
print('Saving trained logistic_regression model  to {}'.format(model_output)) 
f=open(model_output, "wb")
pickle.dump( model1,f)
f.close()